In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/transactions_data.csv')
df.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          object 
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.2+ GB


In [ ]:
# 2. Ubah kolom 'date' ke datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 3. Bersihkan kolom 'amount'
# - hapus $ , koma, spasi
# - ubah (77.00) jadi -77.00
df['amount'] = (
    df['amount']
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)    # hapus $ dan koma
    .str.replace(r'\s+', '', regex=True)      # hapus spasi
    .str.replace(r'^\((.*)\)$', r'-\1', regex=True)  # kurung jadi negatif
).astype(float)

# 4. Normalisasi kolom teks
df['merchant_city'] = df['merchant_city'].fillna("UNKNOWN").str.title()
df['merchant_state'] = df['merchant_state'].fillna("UNKNOWN").str.upper()
df['zip'] = df['zip'].fillna("UNKNOWN").astype(str)

# 5. Kolom errors → kosong = NaN
df['errors'] = df['errors'].replace('', pd.NA)

# 6. Cek hasil cleaning
print(df.info())
print(df.head())
print(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          float64       
 5   use_chip        object        
 6   merchant_id     int64         
 7   merchant_city   object        
 8   merchant_state  object        
 9   zip             object        
 10  mcc             int64         
 11  errors          object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 1.2+ GB
None
        id                date  client_id  card_id  amount           use_chip  \
0  7475327 2010-01-01 00:01:00       1556     2972  -77.00  Swipe Transaction   
1  7475328 2010-01-01 00:02:00        561     4575   14.57  Swipe Transaction   
2  7475329 2010-01-01 00:02:00       1129      102  

In [ ]:
# 7. Buang duplikat berdasarkan id transaksi
df = df.drop_duplicates(subset=['id'])

# 8. Hapus baris kalau ada missing di kolom penting
df = df.dropna(subset=['id', 'date', 'client_id', 'card_id', 'amount'])

print("Setelah cleaning:")
print(df.info())

# 9. Simpan hasil cleaning jadi file CSV utuh
df.to_csv("transactions_clean.csv", index=False)
print("✅ File hasil cleaning disimpan: transactions_clean.csv")

# 10. Split file jadi potongan max 100 MB
import os

max_size = 100 * 1024 * 1024  # 100 MB
rows_per_chunk = 500_000      # coba 500 ribu baris sekali simpan, bisa disesuaikan
i = 1
start = 0
while start < len(df):
    end = start + rows_per_chunk
    chunk = df.iloc[start:end]
    filename = f"transactions_part_{i}.csv"
    chunk.to_csv(filename, index=False)
    size = os.path.getsize(filename)
    print(f"File {filename} dibuat, ukuran {size/1024/1024:.2f} MB")

    # kalau kebesaran, bisa kecilin rows_per_chunk
    start = end
    i += 1

print("✅ Semua file sudah dipisah ≤100MB")

Setelah cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          float64       
 5   use_chip        object        
 6   merchant_id     int64         
 7   merchant_city   object        
 8   merchant_state  object        
 9   zip             object        
 10  mcc             int64         
 11  errors          object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 1.2+ GB
None
✅ File hasil cleaning disimpan: transactions_clean.csv
File transactions_part_1.csv dibuat, ukuran 45.06 MB
File transactions_part_2.csv dibuat, ukuran 45.07 MB
File transactions_part_3.csv dibuat, ukuran 45.06 MB
File transactions_part_4.csv dibuat, ukuran 45.08 MB
File trans

In [ ]:
import pandas as pd

# 1. Baca data
users = pd.read_csv("users_data.csv")

# 2. Drop duplikat
users = users.drop_duplicates(subset=['id'])

# 3. Hapus baris dengan missing value di kolom penting
users = users.dropna(subset=['id', 'gender', 'current_age', 'credit_score'])

# 4. Bersihkan kolom uang → buang $ dan koma
for col in ['per_capita_income', 'yearly_income', 'total_debt']:
    users[col] = (
        users[col]
        .astype(str)
        .str.replace(r'[\$,]', '', regex=True)
        .str.replace(r'\s+', '', regex=True)
    ).astype(float)

# 5. Standardisasi gender
users['gender'] = users['gender'].str.title()  # Male/Female

# 6. Simpan hasil cleaning
users.to_csv("users_clean.csv", index=False)
print("✅ users_clean.csv berhasil dibuat, jumlah data:", len(users))


✅ users_clean.csv berhasil dibuat, jumlah data: 2000


In [ ]:
# 1. Baca data
cards = pd.read_csv("cards_data.csv")

# 2. Drop duplikat
cards = cards.drop_duplicates(subset=['id'])

# 3. Hapus baris dengan missing value di kolom penting
cards = cards.dropna(subset=['id', 'client_id', 'card_type', 'card_number'])

# 4. Bersihkan kolom credit_limit → buang $ dan koma
cards['credit_limit'] = (
    cards['credit_limit']
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .str.replace(r'\s+', '', regex=True)
).astype(float)

# 5. Standardisasi kolom teks
cards['card_brand'] = cards['card_brand'].str.title()
cards['card_type'] = cards['card_type'].str.title()
cards['has_chip'] = cards['has_chip'].str.upper()
cards['card_on_dark_web'] = cards['card_on_dark_web'].str.title()

# 6. Simpan hasil cleaning
cards.to_csv("cards_clean.csv", index=False)
print("✅ cards_clean.csv berhasil dibuat, jumlah data:", len(cards))

✅ cards_clean.csv berhasil dibuat, jumlah data: 6146
